In [1]:
!pip install fastapi nest-asyncio pyngrok uvicorn

     |████████████████████████████████| 54 kB 2.1 MB/s 
     |████████████████████████████████| 745 kB 15.9 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 58 kB 4.2 MB/s 
     |████████████████████████████████| 10.9 MB 1.9 MB/s 
     |████████████████████████████████| 79 kB 7.4 MB/s 
     |████████████████████████████████| 58 kB 3.1 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=abb4e0dc521a94830c973062448d5481bcb2ba8bc39c0eb616db5312a8fd85cd
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [2]:
from __future__ import annotations
import os
import json
from datetime import datetime
from threading import Thread, current_thread, get_ident
from typing import Dict, List, Union
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline


from typing import Dict, List, Optional, Union
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
model=joblib.load('/content/drive/MyDrive/AIM_Task/SVM_Model_final.joblib')

In [4]:
labels_dec={0: 'AE',
 1: 'BH',
 2: 'DZ',
 3: 'EG',
 4: 'IQ',
 5: 'JO',
 6: 'KW',
 7: 'LB',
 8: 'LY',
 9: 'MA',
 10: 'OM',
 11: 'PL',
 12: 'QA',
 13: 'SA',
 14: 'SD',
 15: 'SY',
 16: 'TN',
 17: 'YE'}

In [5]:
def predict(self, texts: List[str]) -> List[Dict]:
    response = []
    if self.model:
        probs = self.model.predict_proba(texts)
        for i, row in enumerate(probs):
            row_pred = {}
            row_pred['text'] = texts[i]
            row_pred['predictions'] = {class_: round(float(prob), 3) for class_, prob in zip(
                self.model_status['classes'], row)}
            response.append(row_pred)
    else:
        raise Exception("No Trained model was found.")
    return response

def get_status(self) -> Dict:
    return self.model_status

def _update_status(self, status: str, classes: List[str] = [], evaluation: Dict = {}) -> None:
    self.model_status['status'] = status
    self.model_status['timestamp'] = datetime.now().isoformat()
    self.model_status['classes'] = classes
    self.model_status['evaluation'] = evaluation

    with open(self.__status_path, 'w+') as file:
        json.dump(self.model_status, file, indent=2)

In [6]:
app = FastAPI()



class TestingData(BaseModel):
    texts: List[str]

class QueryText(BaseModel):
    text: str

class StatusObject(BaseModel):
    status: str
    timestamp: str
    classes: List[str]
    evaluation: Dict

class PredictionObject(BaseModel):
    text: str
    predictions: Dict

class PredictionsObject(BaseModel):
    predictions: List[PredictionObject]


@app.get("/status", summary="Get current model pipeline and paramaters")
def get_status():
    status = model
    return status



@app.post("/predict", summary="Predict single input")
def predict(query_text: QueryText):
    try:
        prediction = labels_dec[model.predict([query_text.text])[0]]
        return prediction
    except Exception as e:
        raise HTTPException(status_code=503, detail=str(e))

@app.post("/predict-batch", summary="predict a batch of sentences")
def predict_batch(testing_data:TestingData):
    try:
        predictions = model.predict(testing_data.texts)
        res=[]
        for p in predictions :
          p=labels_dec[p]
          res.append(p)
        return res
    except Exception as e:
        raise HTTPException(status_code=503, detail=str(e))


@app.get("/")
def home():
    return({"message": "System is up"})

In [7]:
!ngrok authtoken 26NqrTAueDj2zczpSWJPeEc943x_3aJ37uA7kWFhQXFegXQhc

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: http://8681-34-125-29-84.ngrok.io
